In [ ]:
# ! pip install git+https://github.com/speechbrain/speechbrain.git@develop


In [ ]:
print('test')

In [ ]:
import torch
import torchaudio
import torchaudio.transforms as T
import sys
import tqdm

from speechbrain.inference.speaker import EncoderClassifier
# from speechbrain.inference.speaker import SpeakerRecognition


###UPDATE add robustness in inputs
sys.path.append('/om2/user/salavill/misc/voice-speech-metamers/metamers_pipeline/robustness')
from attack_steps import L2Step


ModuleNotFoundError: No module named 'cox'

Compute your speaker embeddings

### Notes about input
- needs to 16 kHz

In [1]:
audio = '/om2/user/amagaro/voice-speech-metamers/metamers_pipeline/kell2018/metamers/psychophysics_wsj400_jsintest_inversion_loss_layer_RS0_I3000_N8/0_SOUND_million/orig.wav'
audio2 = '/om2/user/amagaro/voice-speech-metamers/metamers_pipeline/kell2018/metamers/psychophysics_wsj400_jsintest_inversion_loss_layer_RS0_I3000_N8/1_SOUND_would/orig.wav'

In [2]:
# Get embedding for speaker
model = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")
# signal, fs = torchaudio.load(audio)
# # make sure to resample to appropriate frequency
# resampler = T.Resample(fs, 16000, dtype=signal.dtype)
# signal = resampler(signal)
# embeddings = model.encode_batch(signal)

NameError: name 'EncoderClassifier' is not defined

# Load in the dataset

In [ ]:
# load in sound
signal, fs = torchaudio.load(audio)
# make sure to resample to appropriate frequency
resampler = T.Resample(fs, 16000, dtype=signal.dtype)
signal = resampler(signal)

In [ ]:
target = model.encode_batch(signal)[0][0]

# Initialize noise

In [ ]:
# initialize random noise
NOISE_SCALE = .0000001
im_n_initialized = ((torch.randn_like(signal)) * NOISE_SCALE ).detach().cpu().numpy()

# Define loss

In [ ]:
class InversionLossLayer_ecapa(torch.nn.Module):
    """Loss used for most metamer generation experiments"""
    def __init__(self, normalize_loss=False):
        super(InversionLossLayer_ecapa, self).__init__()
        self.normalize_loss = normalize_loss

    def forward(self, model, inp, targ):
        # CHange to get ecapa model embeddings
        all_outputs = model.encode_batch(inp)
        rep = all_outputs[0][0]
        if self.normalize_loss:
            loss = torch.div(torch.norm(rep - targ, dim=1), torch.norm(targ, dim=1))
        else:
            loss = torch.norm(rep - targ, dim=1)
        return loss, None

In [ ]:
# params
normalize_loss = False ###UPDATE: idk

In [ ]:
calc_loss = InversionLossLayer_ecapa(normalize_loss = normalize_loss)

# Iterate through and update Noise

- might need to constrain the update of noise

In [ ]:
# params
# synth_kwargs = {
#     'custom_loss': custom_synthesis_losses.LOSSES[LOSS_FUNCTION](layer_to_invert, normalize_loss=True),
#     'constraint':'2',
#     'eps':100000,
#     'step_size': step_size,
#     'iterations': ITERATIONS,
#     'do_tqdm': False,
#     'targeted': True,
#     'use_best': False
# }
eps = 100000
step_size = 1


random_start = 0 #UPDATE: IDK
do_tqdm = 0 # import 

targeted = 1
m = -1 if targeted else 1 #UPDATE: idk

dataset_min_value = signal.min_value
dataset_max_value = signal.max_value

use_best = 0 #

iterations = 100 # in the pipeline Janelle uses 3000
return_image = 1

est_grad = None

In [ ]:
# Iter n times
# set noise as parameters to update

# update these parameters 
step = L2Step(eps=eps, orig_input=signal, step_size=step_size,
            min_value=dataset_min_value, max_value=dataset_max_value)


In [ ]:
# Main function for making adversarial examples
def get_adv_examples(x):
    # Random start (to escape certain types of gradient masking)
    if random_start:
        x = step.random_perturb(x)

    iterator = range(iterations)
    if do_tqdm: iterator = tqdm(iterator)

    # Keep track of the "best" (worst-case) loss and its
    # corresponding input
    best_loss = None
    best_x = None

    # A function that updates the best loss and best input
    def replace_best(loss, bloss, x, bx):
        if bloss is None:
            bx = x.clone().detach()
            bloss = loss.clone().detach()
        else:
            replace = m * bloss < m * loss
            bx[replace] = x[replace].clone().detach()
            bloss[replace] = loss[replace]

        return bloss, bx

    # PGD iterates
    for _ in iterator:
        x = x.clone().detach().requires_grad_(True)
        losses, out = calc_loss(step.to_image(x), signal)
        assert losses.shape[0] == x.shape[0], \
                'Shape of losses must match input!'

        loss = torch.mean(losses)

        if step.use_grad:
            if est_grad is None:
                grad, = torch.autograd.grad(m * loss, [x])
            # else:
            #     f = lambda _x, _y: m * calc_loss(step.to_image(_x), _y)[0]
            #     grad = helpers.calc_est_grad(f, x, target, *est_grad)
        else:
            grad = None

        with torch.no_grad():
            args = [losses, best_loss, x, best_x]
            best_loss, best_x = replace_best(*args) if use_best else (losses, x)

            x = step.step(x, grad)
            x = step.project(x)
            if do_tqdm: iterator.set_description("Current loss: {l}".format(l=loss))

    # Save computation (don't compute last loss) if not use_best
    if not use_best: 
        ret = x.clone().detach()
        return step.to_image(ret) if return_image else ret

    losses, _ = calc_loss(step.to_image(x), target)
    args = [losses, best_loss, x, best_x]
    best_loss, best_x = replace_best(*args)
    return step.to_image(best_x) if return_image else best_x


In [ ]:
xadv = get_adv_examples(im_n_initialized)
this_loss = calc_loss(model, xadv, target.clone())

In [ ]:
for i in range(iterations):

    im_n = xadv
    # step.step_size = step.step_size/2
    # get metamer
    xadv = get_adv_examples(im_n)
    # calculate new loss
    this_loss = calc_loss(model, xadv, target.clone())
    if i%10 == 0:
        print(f'Iteration: {i}, Loss: {this_loss}')

In [ ]:
# play metamer
xadv


Perform Speaker Verification

In [ ]:
# Get embedding for speaker
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")
signal, fs = torchaudio.load(audio)
# make sure to resample to appropriate frequency
resampler = T.Resample(fs, 16000, dtype=signal.dtype)
signal = resampler(signal)
embeddings = classifier.encode_batch(signal)

In [32]:
# verify if 2 speakers are the same
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_models/spkrec-ecapa-voxceleb")
# score, prediction = verification.verify_files(audio, audio) # Same Speaker
# print(score, prediction)

score, prediction = verification.verify_files(audio, audio2) # Different Speakers
print(score, prediction)


tensor([1.]) tensor([True])


In [36]:
import scipy
scipy.io.wavfile.read(audio)

(20000,
 array([ 0.00759776,  0.00406835, -0.00337481, ..., -0.01439899,
        -0.0056413 ,  0.01752515], dtype=float32))

In [27]:
embeddings

tensor([[[ 13.6807, -21.5077,  16.5698,  27.3773,  20.9341,  -2.6867,  47.9093,
           10.9244,  43.0350, -29.4488, -50.7251,  23.6661,  36.2392,   5.4932,
            4.8596, -14.1890,  -7.8909,   3.0897,  40.3050, -14.0334, -18.2770,
          -19.1589,  33.1250, -16.0443, -15.0739,  -6.6625, -21.3170,  -4.1494,
           14.0383, -15.5008, -30.3342,  41.4378,  25.2998,   7.5653,  15.8887,
           10.5355, -24.6812, -25.7595, -19.5165,   2.4563,  23.4598,  12.1603,
          -14.5642,  -8.9963,   8.4244, -45.2159,  23.8665,  12.0209,  -9.0004,
           29.5747,  15.1678,  17.9769, -12.2870,   0.7903,  10.2497,  11.3309,
           -9.3533,  -0.3147,   7.0483, -10.6227,   5.6716, -13.1225, -24.9788,
            6.1509,  48.9741, -20.4996,  -0.8317, -11.3773,  27.6694, -15.0420,
            8.6980, -20.4009,  15.0508,  13.3836, -20.7181,  -4.8599, -22.3480,
          -35.1690, -11.5540,  14.0490, -29.5636,  29.2453,  21.0988,  40.4526,
           50.2919, -13.8887, -21.6431, 

In [53]:
list(classifier._modules)

['mods']

In [55]:
classifier._modules['mods']['classifier']

Classifier(
  (blocks): ModuleList()
)

In [39]:
embeddings


tensor([[[-1.1256e+01, -4.4376e+00,  5.4710e+00, -4.4777e-01,  9.3813e+00,
          -1.7214e+01,  2.0267e+01, -6.4316e+00,  1.1311e+00, -6.9549e+00,
          -5.6514e+01,  1.5606e+01,  7.6293e+00,  9.4477e+00,  5.0318e+00,
           6.7749e+00,  2.2657e+01,  6.6792e+00,  1.2737e+01,  4.7252e+00,
          -1.1207e+00, -3.8659e+01,  5.9548e+01,  1.3946e+01, -2.6766e+01,
          -1.5729e+01, -6.6083e+00, -2.0886e+01, -3.2234e+01, -3.1146e+01,
           5.3255e+00,  1.0481e+01,  2.7337e+01,  2.0878e+01,  1.0505e+01,
          -5.3502e+00,  1.4651e+01, -3.3391e+00, -1.6250e+00, -3.7518e+01,
           3.7116e+01,  2.6891e+01, -3.3418e+01,  1.3859e+01,  7.2535e+00,
          -3.0828e+00, -1.5202e+01,  8.6623e+00,  2.5144e+01,  3.2541e+01,
           1.8105e+01,  4.8451e+00, -1.4034e+01,  3.1070e+01,  8.0962e+01,
          -3.7654e+00, -3.5467e+01,  1.6404e+00, -4.7164e+01,  4.4524e+00,
          -4.2345e+00, -1.4514e+01,  6.2556e-02,  4.1281e+00,  4.3622e+01,
           1.6985e+01, -1

In [15]:
score, prediction

(tensor([1.]), tensor([True]))